## This code is for efficient apiori algorithm

In [1]:
import time
import itertools

#   This function generates the first set of candidate itemsets using the dataset
def generateC1(input_data):
    item_dict = {}
    final_Set = []
    for data in input_data:
        for item in data:
            if item not in item_dict:
                 item_dict[item] = 1
            else:
                 item_dict[item] = item_dict[item] + 1
    for key in item_dict:
        current_Array = []
        current_Array.append(key)
        final_Set.append(current_Array)
        final_Set.append(item_dict[key])
        current_Array = []
    return final_Set

#   This function creates Frequent item sets by taking candidate sets as input
#   At the end, this function calls generateCandidatSets by feeding the output of the
#   current function as the input of the other function

def generateFrequentItemSet(list_of_Candidates, no_of_transac, minsup, input_data, parent_frequent_array):
    list_of_frequentitems = []
    for i in range(len(list_of_Candidates)):
        if i%2 != 0:
            support = (list_of_Candidates[i] * 1.0 / no_of_transac) * 100
            if support >= minsup:
                list_of_frequentitems.append(list_of_Candidates[i-1])
                list_of_frequentitems.append(list_of_Candidates[i])
            else:
                list_of_eliminated_items.append(list_of_Candidates[i-1])

    for k in list_of_frequentitems:
        parent_frequent_array.append(k)

    if len(list_of_frequentitems) == 2 or len(list_of_frequentitems) == 0:
        #print("This will be returned")
        returnArray = parent_frequent_array
        return returnArray

    else:
        generateCandidateSets(input_data, list_of_eliminated_items, list_of_frequentitems, no_of_transac, minsup)

        
#   This function takes frequent sets as input and generates Candidate sets. 
# Finally, this function calls another function that generates Frequent Item Sets by using the output of the current function 
# as the input of the other function's function.


def generateCandidateSets(input_data, list_of_eliminated_items, list_of_frequentitems, no_of_transac, minsup):
    onlyElements = []
    arrayAfterCombinations = []
    candidateSetArray = []
    for i in range(len(list_of_frequentitems)):
        if i%2 == 0:
            onlyElements.append(list_of_frequentitems[i])
    for item in onlyElements:
        current_CombinationArray = []
        k = onlyElements.index(item)
        for i in range(k + 1, len(onlyElements)):
            for j in item:
                if j not in current_CombinationArray:
                    current_CombinationArray.append(j)
            for m in onlyElements[i]:
                if m not in current_CombinationArray:
                    current_CombinationArray.append(m)
            arrayAfterCombinations.append(current_CombinationArray)
            current_CombinationArray = []
    sortedCombinationArray = []
    uniqueCombinationArray = []
    for i in arrayAfterCombinations:
        sortedCombinationArray.append(sorted(i))
    for i in sortedCombinationArray:
        if i not in uniqueCombinationArray:
            uniqueCombinationArray.append(i)
    arrayAfterCombinations = uniqueCombinationArray
    for item in arrayAfterCombinations:
        count = 0
        for transaction in input_data:
            if set(item).issubset(set(transaction)):
                count = count + 1
        if count != 0:
            candidateSetArray.append(item)
            candidateSetArray.append(count)
    generateFrequentItemSet(candidateSetArray, no_of_transac, minsup, input_data, parent_frequent_array)
    

#   This function takes all the frequent sets as the input and generates Association Rules


def generateAssociationRule(freqSet):
    associationRule = []
    for item in freqSet:
        if isinstance(item, list):
            if len(item) != 0:
                length = len(item) - 1
                while length > 0:
                    combinations = list(itertools.combinations(item, length))
                    current = []
                    LHS = []
                    for RHS in combinations:
                        LHS = set(item) - set(RHS)
                        current.append(list(LHS))
                        current.append(list(RHS))
                        #print(current)
                        associationRule.append(current)
                        current = []
                    length = length - 1
    return associationRule

#   This function creates the final output of the algorithm by taking Association Rules as the input
def aprioriOutput(rules, input_data, minsup, min_conf):
    final_AprioriOutput = []
    for rule in rules:
        sup_of_x = 0
        sup_of_x_in_percent = 0
        sup_of_xandy = 0
        sup_of_xandy_in_percent = 0
        for transaction in input_data:
            if set(rule[0]).issubset(set(transaction)):
                sup_of_x = sup_of_x + 1
            if set(rule[0] + rule[1]).issubset(set(transaction)):
                 sup_of_xandy = sup_of_xandy + 1
        sup_of_x_in_percent = (sup_of_x * 1.0 / no_of_transac) * 100
        sup_of_xandy_in_percent = (sup_of_xandy * 1.0 / no_of_transac) * 100
        confidence = (sup_of_xandy_in_percent / sup_of_x_in_percent) * 100
        if confidence >= min_conf:
            supportOfXAppendString = "Support Of X: " + str(round(sup_of_x_in_percent, 2))
            supportOfXandYAppendString = "Perc. Support of X & Y: " + str(round(sup_of_xandy_in_percent))
            confidenceAppendString = "Perc. Confidence: " + str(round(confidence))

            final_AprioriOutput.append(supportOfXAppendString)
            final_AprioriOutput.append(supportOfXandYAppendString)
            final_AprioriOutput.append(confidenceAppendString)
            final_AprioriOutput.append(rule)

    return final_AprioriOutput

In [2]:
fileName = "no_commas.txt"
allFrequentItemSets = []
list_of_eliminated_items = []
no_of_transac = 0
parent_frequent_array = []
something = 0

minsup = 100*float(input(" Enter a minimum Support value :"))
min_conf = 100*float(input(" Enter a minimum Confidence value : "))

print()

nonFrequentSets = []
current_FrequentItemSets = []
input_data = []


begin=time.time()

#   Reading the data file line by line
with open(fileName,'r') as fp:
    lines = fp.readlines()

for line in lines:
    line = line.rstrip()
    input_data.append(line.split(","))

no_of_transac = len(input_data)

firstCandidateSet = generateC1(input_data)

frequentItemSet = generateFrequentItemSet(firstCandidateSet, no_of_transac, minsup, input_data, parent_frequent_array)

associationRules = generateAssociationRule(parent_frequent_array)

AprioriOutput = aprioriOutput(associationRules, input_data, minsup, min_conf)


counter = 1
if len(AprioriOutput) == 0:
    print("There are no association rules for this support and confidence.")
else:
    for i in AprioriOutput:
        if counter == 4:
            print(str(i[0]) + "------>" + str(i[1]))
            counter = 0
        else:
            print(i, end='  ')
        counter = counter + 1
        
end=time.time()


print("\nTime taken by our algorithm is :",end-begin)

 Enter a minimum Support value :0.05
 Enter a minimum Confidence value : 0.2

Support Of X: 17.2  Perc. Support of X & Y: 10  Perc. Confidence: 56  ['32']------>['39']
Support Of X: 17.2  Perc. Support of X & Y: 9  Perc. Confidence: 53  ['32']------>['48']
Support Of X: 57.48  Perc. Support of X & Y: 12  Perc. Confidence: 20  ['39']------>['38']
Support Of X: 17.69  Perc. Support of X & Y: 12  Perc. Confidence: 66  ['38']------>['39']
Support Of X: 17.69  Perc. Support of X & Y: 9  Perc. Confidence: 51  ['38']------>['48']
Support Of X: 16.95  Perc. Support of X & Y: 13  Perc. Confidence: 76  ['41']------>['39']
Support Of X: 57.48  Perc. Support of X & Y: 13  Perc. Confidence: 23  ['39']------>['41']
Support Of X: 47.79  Perc. Support of X & Y: 33  Perc. Confidence: 69  ['48']------>['39']
Support Of X: 57.48  Perc. Support of X & Y: 33  Perc. Confidence: 58  ['39']------>['48']
Support Of X: 47.79  Perc. Support of X & Y: 10  Perc. Confidence: 21  ['48']------>['41']
Support Of X: 16